In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using LinearAlgebra, Plots
using ProgressMeter
import ForwardDiff as FD
import Convex as cvx
import ECOS

In [ ]:
# """
# convert from cartesian (x,y) to polar (r,θ) coordinates.
# """
# function car2pol(x::Float64, y::Float64)
#     r::Float64 = √(x^2 + y^2)
#     θ::Float64 = atan(y, x)
#     return r, θ
# end;

function wrap2pi(θ)
    return θ - 2*π*floor((θ+π)/(2*π))
end;

function mat_from_vec(X::Vector{Vector{Float64}})::Matrix
    # convert a vector of vectors to a matrix 
    Xm = hcat(X...)
    return Xm 
end;

function vec_from_mat(Xm::Matrix)::Vector{Vector{Float64}}
    # convert a matrix into a vector of vectors 
    X = [Xm[:,i] for i = 1:size(Xm,2)]
    return X 
end;

### Variable Definitions

#### State Variables:
 - $(x_o, y_o)$ - global position
 - $\theta$ - global orientation
 - $v$ - velocity
 - $\dot{\theta}$ - angular velocity

#### Control Variables:
 - $\delta_r$ - rudder angle
 - $\delta_s$ - sail angle

#### Wind and Force Variables:
 - $a_{tw}$ - true wind speed
 - $\psi_{tw}$ - true wind direction

#### Parameters:
 - $p_1$ - drift coefficient
 - $p_2$ - tangential friction
 - $p_3$ - angular friction
 - $p_4$ - sail lift
 - $p_5$ - rudder lift
 - $p_6$ - distance to sail CoE (center of effort)
 - $p_7$ - distance to mast
 - $p_8$ - distance to rudder
 - $p_9$ - mass of boat
 - $p_{10}$ - moment of inertia
 - $p_{11}$ - rudder break coefficient


### System Model

#### State and Input Arrays:
$$ \begin{align}
x &= \begin{bmatrix} x_{b} & y_{b} & \theta & v & \dot{\theta} \end{bmatrix} ^T \\
u &= \begin{bmatrix} \delta_r & \delta_s \end{bmatrix} ^T
\end{align} $$

#### True Wind:
$$ \begin{align}
\dot{x}_{tw} &= w_x(x_b, y_b) \\
\dot{y}_{tw} &= w_y(x_b, y_b) \\
a_{tw} &= \sqrt{\dot{x}_{tw}^2 + \dot{y}_{tw}^2}  \\
\psi_{tw} &= \arctan{(\frac{\dot{y}_{tw}}{\dot{x}_{tw}})} \\
\end{align} $$

#### Apparent Wind:
$$ \begin{align}
\dot{x}_{aw} &= a_{tw} \cos{(\psi_{tw} - \theta)} - v \\
\dot{y}_{aw} &= a_{tw} \sin{(\psi_{tw} - \theta)} \\
a_{aw} &= \sqrt{\dot{x}_{aw}^2 + \dot{y}_{aw}^2}  \\
\psi_{aw} &= \arctan{(\frac{\dot{y}_{aw}}{\dot{x}_{aw}})} \\
\end{align} $$

#### Sail and Rudder Forces:
$$ \begin{align}
g_s &= p_4 a_{aw} \sin{(\delta_s - \psi_{aw})} \\
g_r &= p_5 v^2 \sin{\delta_r}
\end{align} $$

#### Dynamics

$$ \begin{align}
\dot{x} = f(x, u, w) &= 
    \begin{bmatrix}
        v \cos{\theta} + p_1 a_{tw} \cos{\psi_{tw}} \\
        v \sin{\theta} + p_1 a_{tw} \sin{\psi_{tw}} \\
        \dot{\theta} \\
        \frac{g_s \sin{\delta_s} - g_r p_{11} \sin{\delta_r} - p_2 v^2}{p_9} \\
        \frac{g_s (p_6 - p_7 \cos{\delta_s}) - g_r p_8 \cos{d_r} - p_3 \dot{\theta} v}{p_{10}}
    \end{bmatrix}
\end{align} $$

In [ ]:
"""
true wind velocity (vector field)
"""
function true_wind(xₒ,yₒ)
    ẋ_tw = 1.
    ẏ_tw = 0.
    return ẋ_tw, ẏ_tw
end

"""
dynamics
"""
function dynamics(x, u, p)
    xₒ, yₒ, θ, v, θ̇ = x
    δᵣ, δₛ = u
    
    # true wind
    ẋ_tw, ẏ_tw = true_wind(xₒ,yₒ)
    # a_tw, ψ_tw = car2pol(ẋ_tw, ẏ_tw)
    a_tw = √(ẋ_tw^2 + ẏ_tw^2)
    ψ_tw = atan(ẏ_tw, ẋ_tw)
    
    # apparent wind
    ẋ_aw = a_tw * cos(ψ_tw - θ) - v
    ẏ_aw = a_tw * sin(ψ_tw - θ)
    # a_aw, ψ_aw = car2pol(ẋ_aw, ẏ_aw)
    a_aw = √(ẋ_aw^2 + ẏ_aw^2)
    ψ_aw = atan(ẏ_aw, ẋ_aw)
    
    # forces
    gₛ = p[4] * a_aw * sin(δₛ - ψ_aw)
    gᵣ = p[5] * v^2 * sin(δᵣ)

    # dynamics
    ẋ = [v*cos(θ) + p[1]*a_tw*cos(ψ_tw),
         v*sin(θ) + p[1]*a_tw*sin(ψ_tw),
         θ̇,
         (gₛ*sin(δₛ) - gᵣ*p[11]*sin(δᵣ) - p[2]*v^2)/(p[9]),
         (gₛ*(p[6] - p[7]*cos(δₛ)) - gᵣ*p[8]*cos(δᵣ) - p[3]*θ̇*v)/(p[10])]

    return ẋ
end;

### Linearization

$$ \begin{align}
A_c &= \frac{\delta f}{\delta x} \\
B_c &= \frac{\delta f}{\delta u}
\end{align} $$

In [ ]:
"""
linear dynamics about a point
"""
function linearized_dynamics(x, u, p)
    Ac = FD.jacobian(_x -> dynamics(_x, u, p), x)
    Bc = FD.jacobian(_u -> dynamics(x, _u, p), u)

    return Ac, Bc
end;

### Discretization

In [ ]:
"""
discretize a linear system using the matrix exponential
"""
function discretize(Ac, Bc, dt)
    nx, nu = size(Bc)

    # continuous-time block for [x u] state
    Blockc = [Ac Bc;
              zeros(nu,nx+nu)]
    
    # discrete-time block for [x u] state
    Block = exp(Blockc*dt)
    
    A = Block[1:nx, 1:nx]
    B = Block[1:nx, nx+1:nx+nu]
    
    return A, B
end;

### Integration

In [ ]:
function rk4(x, u, p, dt)
    k1 = dt*dynamics(x, u, p)
    k2 = dt*dynamics(x + k1/2, u, p)
    k3 = dt*dynamics(x + k2/2, u, p)
    k4 = dt*dynamics(x + k3, u, p)

    x_next = x + (k1 + 2*k2 + 2*k3 + k4)/6
    x_next[2] = wrap2pi(x_next[2])
    
    return x_next
end;

### Trajectory Planning (Direct Collocation)

In [ ]:
#TODO

### Trajectory Tracking (TV-LQR)

In [ ]:
#TODO

### Old

In [ ]:
p = [0.03, 40, 6000, 200, 1500, 0.5, 0.5, 2, 300, 400, 0.2]
Δt = 0.1
nx = 5
nu = 2

x0 = [0.0, 0.0, 0.0, 0.0, 0.0]
x_goal = [100.0, 0.0, 0.0, 0.0, 0.0]

Q = diagm([10.0, 10.0, 1.0, 1.0, 1.0])
R = diagm([1.0, 1.0])

u_min = [-5*π/12, -5*π/12]
u_max = [5*π/12, 5*π/12]

N = 1000
t_vec = 0:Δt:((N-1)*Δt)

X_sim = [zeros(nx) for i = 1:N_sim]
X_sim[1] = x0
X_sim[2] = x0
U_sim = [zeros(nu) for i = 1:N_sim-1];
U_sim[1] = [0.01, 0.01]
U_sim[2] = [0.01, 0.01]

In [ ]:
@showprogress "simulating" for i = 2:N_sim-1
    A, B = discretize(linearized_dynamics(X_sim[i], U_sim[i-1], p)..., Δt)

    U_sim[i] = [0,0]
    X_sim[i+1] = rk4(X_sim[i], U_sim[i], p, Δt)
end

In [27]:
Xm = mat_from_vec(X_sim)
Um = mat_from_vec(U_sim);

display(plot(Xm[1,:],Xm[2,:],title = "Trajectory",
        xlabel = "x position (m)", ylabel = "y position (m)"))

display(plot(t_vec,Xm[1:2,:]',title = "Positions",
        xlabel = "time (s)", ylabel = "positions (m)",
        label = ["x" "y"]))

# display(plot(t_vec,Xm[4:6,:]',title = "Velocities",
#         xlabel = "time (s)", ylabel = "velocity (m/s)",
#         label = ["x" "y" "z"]))

display(plot(t_vec[1:end-1],Um[1:2,:]',title = "Control",
        xlabel = "time (s)", ylabel = "angle (rad)",
        label = ["rudder" "sail"]))